# Projet AlgoDS : école CentraleSupelec 2021-2022

![CentraleSupelec Logo](https://www.centralesupelec.fr/sites/all/themes/cs_theme/medias/common/images/intro/logo_nouveau.jpg)

Here the aim is to study the inout csv which nothing else than the input.csv file and output.csv file merged. Here we would like to explore the dataset and discover pattern in this timeseries. Whereas the ENS challenge was about timeseries classification, here we would like to do time series forecasting knowing the class of the time series. In other words, we have to predict the comsumption at d+2 knowing the consumer. 

# Modules and stuffs

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px

# Data exploration

## Dataset $\mathcal{D}$ in short

In [ ]:
df = pd.read_csv("../dataset/inout.csv")

In [ ]:
print(df.shape)
print(df.describe())

In [ ]:
print(df.columns)

In [ ]:
print(df.info(memory_usage='deep'))## dis le vrai usage de mémoire du df 

In [ ]:
print(df.memory_usage(deep=True))## stockage par série

In [ ]:
plt.title("Data Type")
df.dtypes.value_counts().plot.pie()# petit camenbert des différents type du dataframe
plt.show()

In [ ]:
print((100*df.isna().sum()/df.shape[0]).sort_values(ascending=False))# voir le pourcentage de données manqauntes

In [ ]:
for col in df.select_dtypes(['int','float']):## tu peux aussi faire df.select_dtypes(exclude="object") par exemple
    plt.title(f"{col} distribution")
    sns.histplot(df[col])
    plt.grid()
    plt.show()

We can cleary see that some distrubution look the very same, namely (consommation totale de HTA and consommation totale de HTAtéléversé à la courbe de charge) or (production décentralisée totale et production décentralisée totale téléversée à la courbe de charge), (consommation PME-PMI profilée et consommation résidentielle profilée) to cite only them. 

In [ ]:
plt.figure(figsize=(12,8))
plt.title("Correlation map")
sns.heatmap(df.corr())
plt.show()

There is many things to say with this correlation map. 
* Pertes modélisées et Consommation totale are strongly correlated which is normal
* Injection totale is strongly correlated with Pertes modélisées, Consommation totale, consommation totale profilée and more generally with every consumption variables, but also with soutirage net vers autres GRD and res11 base and pro1Base
* RES1_ et RES2_HP are the major contributor to total enery consumption. 
* Production totale éolienne et refoulement 
* There is a strong negativ correlation <-0.75 between temperature and total consumption. 
* Production décentralisée and production photovoltaïque are strongly correlated almost 1. 
* RES2_HP seems to be correleted with RES1_BASE and RES11_BASE
* RES2_HC and RES2_PRO strongly correlated.

Can be augmented.

We have to keep those classes apart RES1_BASE, PRO1_BASE, RES2_HP, PRO2_HP : each of those contain a value which correspond to the consumption I supposed. 

Warning hours ! 

In [80]:
df=df[0:65130]

In [81]:
df_indexed=df.set_index("Horodate_UTC")
df_indexed.index = pd.to_datetime(df.set_index("Horodate_UTC").index)
df_indexed["Year"]=df_indexed.index.year
df_indexed["Month"]=df_indexed.index.month
df_indexed["Day"]=df_indexed.index.day
df_indexed["week_day"]=df_indexed.index.weekday

In [82]:
min_year=df_indexed.index.year.min()
max_year=df_indexed.index.year.max()
print("min year", df_indexed.index.year.min())
print("max year",df_indexed.index.year.max())
colormap=plt.cm.gist_ncar
colost=[colormap(i) for i in np.linspace(0,0.95,1+max_year-min_year)]## make a nice colorbar

min year 2013
max year 2017


In [83]:
for i in range(min_year,max_year+1,1):
    print(f"nb measures of year {i} : {len(df_indexed[df_indexed['Year']==i])}")

nb measures of year 2013 : 3796
nb measures of year 2014 : 17520
nb measures of year 2015 : 17520
nb measures of year 2016 : 17568
nb measures of year 2017 : 8726


In [84]:
to_be_predicted = df.columns[-7:]

In [ ]:
for column in to_be_predicted :
    plt.figure(figsize=(12,8))
    for i in range(min_year,max_year+1,1):
        plt.plot(df_indexed[df_indexed["Year"]==i][column].values,color=colost[i-min_year],label=i,alpha=0.6)
    plt.title(f"{column} over year")
    plt.legend()
    plt.xlabel("date")
    plt.ylabel(f"{column}")
    plt.grid()
    plt.show()

In [134]:
import plotly.graph_objects as go 

In [ ]:
fig = go.Figure()
fig.add_

In [136]:
for contract in to_be_predicted: 
    df_indexed["Month_angle"]=((df_indexed["Month"])*360/12)
    df_indexed["val"]=df_indexed.groupby(["Month","Year"])[contract].transform("mean")
    fig = px.line_polar(df_indexed.drop_duplicates(subset=["Month","Year","val"]), r="val", theta="Month_angle",
                       color="Year",
                       color_discrete_sequence=px.colors.sequential.Plasma_r,
                       title=f"Contract type : {contract}",
                       line_close=True,
                       labels=["Month"],
                       )
    fig.show()